<a href="https://colab.research.google.com/github/toot09/posItemClassification/blob/master/POSClassification_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s * https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 46.9MB/s 
     |████████████████████████████████| 450kB 35.6MB/s 
     |████████████████████████████████| 92kB 8.8MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ curl -s d2v_1_1000_400_CJ.model d2v_2_1000_400_CJ.model d2v_3_1000_400_CJ.model d2v_4_1000_400_CJ.model d2v_5_1000_400_CJ.model sample_data https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ bash -x
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|m

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt
import pandas as pd
import numpy as np
from konlpy.tag import Mecab
mecab = Mecab()
from konlpy.tag import Okt
okt = Okt()

In [ ]:
df_train = pd.read_csv('./POS_ITEM_CJ.csv', error_bad_lines=False)

b'Skipping line 186: expected 2 fields, saw 3\n'


In [ ]:
df_train.head()

,CD,NM
0,3190550012700000254,A)쁘띠첼밀감110g
1,3190550012700000257,A)쁘띠첼포도110g
2,3190550012700000254,A)쁘띠첼밀감90g
3,3170510011400000229,볼로냐스파게티2인625G_남원/냉장
4,1030120003100000058,튀김전용유0.5L(리뉴얼)/상온


In [ ]:
df_train['CD'] = df_train['CD'].astype(str)

In [ ]:
df_train.loc[df_train['CD'].str.len()==19, 'CD'] = '0'+df_train['CD']

In [ ]:
print(df_train['CD'].str.len().max())
print(df_train['CD'].str.len().min())

20
20


In [ ]:
df_train.head()

,CD,NM
0,03190550012700000254,A)쁘띠첼밀감110g
1,03190550012700000257,A)쁘띠첼포도110g
2,03190550012700000254,A)쁘띠첼밀감90g
3,03170510011400000229,볼로냐스파게티2인625G_남원/냉장
4,01030120003100000058,튀김전용유0.5L(리뉴얼)/상온


In [ ]:
str = '01010010000100000001'
print(str[:2])
print(str[:4])
print(str[:7])
print(str[:12])
print(str[:])

01
0101
0101001
010100100001
01010010000100000001


In [ ]:
df_train['CD_1'] = df_train['CD'].str.slice(start=0, stop=2)
df_train['CD_2'] = df_train['CD'].str.slice(start=0, stop=4)
df_train['CD_3'] = df_train['CD'].str.slice(start=0, stop=7)
df_train['CD_4'] = df_train['CD'].str.slice(start=0, stop=12)
df_train['CD_5'] = df_train['CD']

In [ ]:
df_train['NM'] = df_train['NM'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")#한글과 공백 제외(^ == not)
df_train[:10]

,CD,NM,CD_1,CD_2,CD_3,CD_4,CD_5
0,03190550012700000254,쁘띠첼밀감,03,0319,0319055,031905500127,03190550012700000254
1,03190550012700000257,쁘띠첼포도,03,0319,0319055,031905500127,03190550012700000257
2,03190550012700000254,쁘띠첼밀감,03,0319,0319055,031905500127,03190550012700000254
3,03170510011400000229,볼로냐스파게티인남원냉장,03,0317,0317051,031705100114,03170510011400000229
4,01030120003100000058,튀김전용유리뉴얼상온,01,0103,0103012,010301200031,01030120003100000058
5,01030120003100000058,튀김전용유입상온,01,0103,0103012,010301200031,01030120003100000058
6,03150440010200000204,동그랑땡,03,0315,0315044,031504400102,03150440010200000204
7,02080240006200000129,햇반작은공기입상온,02,0208,0208024,020802400062,02080240006200000129
8,02050180004500000094,맛소금,02,0205,0205018,020501800045,02050180004500000094
9,03150440009900000201,백설숯불구이맛김밥햄냉장,03,0315,0315044,031504400099,03150440009900000201


In [ ]:
df_train['NM'].replace('', np.nan, inplace=True) #위 정제후 빈칸이 되었으면 null(np.nan numpy의 null형태)로 replace
print(df_train.isnull().sum())
df_train = df_train.dropna(how='any')
print("Dropna")
print(df_train.isnull().sum())

CD       0
NM      10
CD_1     0
CD_2     0
CD_3     0
CD_4     0
CD_5     0
dtype: int64
Dropna
CD      0
NM      0
CD_1    0
CD_2    0
CD_3    0
CD_4    0
CD_5    0
dtype: int64


In [ ]:
def spacing_okt(wrongSentence):
    tagged = okt.pos(wrongSentence)
    corrected = ""
    for i in tagged:
        if i[1] in ('Josa', 'PreEomi', 'Eomi', 'Suffix', 'Punctuation'):
            corrected += i[0]
        else:
            corrected += " "+i[0]
    if corrected[0] == " ":
        corrected = corrected[1:]
    return corrected

In [ ]:
dicts = []
dicts.append({})
for n in range(1,6):
  dicts.append({})
  print('Class {} processing'.format(n))
  for i in range(len(df_train)):
      try:
        if df_train.loc[i, 'CD_'+str(n)] not in dicts[n].keys():
          dicts[n][df_train.loc[i, 'CD_'+str(n)]] = ""  
        dicts[n][df_train.loc[i, 'CD_'+str(n)]] = dicts[n][df_train.loc[i, 'CD_'+str(n)]]+" "+spacing_okt(df_train.loc[i, 'NM'])
      except:
        pass

Class 1 processing
Class 2 processing
Class 3 processing
Class 4 processing
Class 5 processing


In [ ]:
datas = []
labels = []
# dummy(0th class)
datas.append([])
labels.append([])
for n in range(1,len(dicts)):
  print("{}th data, labels".format(n))
  datas.append([])
  labels.append([])
  for key, val in dicts[n].items():
      datas[n].append(val)
      labels[n].append(key)

1th data, labels
2th data, labels
3th data, labels
4th data, labels
5th data, labels


In [ ]:
tagged_datas = []
tagged_datas.append([])
for n in range(1,6):
  print('{}th len equal : {}'.format(n,len(labels[n])==len(datas[n])))
  tagged_datas.append([])
  tagged_datas[n] = [TaggedDocument(words=okt.morphs(_d), tags=[str(i)]) for i, _d in enumerate(datas[n])]

1th len equal? : True
2th len equal? : True
3th len equal? : True
4th len equal? : True
5th len equal? : True


In [ ]:
for i in tagged_datas[1]:
  print(i)

TaggedDocument(['쁘띠첼', '밀감', '쁘띠첼', '포도', '쁘띠첼', '밀감', '볼로냐', '스파게티', '인', '남원', '냉장', '동그랑땡', '백설', '숯불', '구이', '맛', '김밥', '햄', '냉장', '쁘띠첼', '밀감', '쁘띠첼', '포도', '쁘띠첼', '복숭아', '허니', '머스타드', '소스', '행복한', '콩가', '지', '국산', '콩', '찌개', '냉장', '가', '쓰오', '우동', '한', '그릇', '입', '용기', '냉장', '쁘띠첼', '요', '거', '젤리', '밀감', '쁘띠첼', '요', '거', '젤리', '딸기', '쁘띠첼', '요', '거', '젤리', '화이트', '코코', '가', '쓰', '오메', '밀생', '면', '냉장', '맥스', '봉', '치즈', '개', '냉장', '맥스', '봉', '치즈', '낱개', '냉장', '다담', '순두부찌개', '양념', '냉장', '다담', '냉이', '된장찌개', '양념', '냉장', '튀김', '우동', '인분', '냉장', '불고기', '맛햄', '하', '이', '푸드', '텍', '냉장', '켄터키', '후', '랑', '크', '입', '홈', '조리', '스파게티', '인분', '냉장', '쁘띠첼', '요', '거', '젤리', '복숭아', '백설', '비엔나', '비엔나', '다담', '된장찌개', '양념', '냉장', '년', '가', '쓰오우', '동인', '분', '인삼', '한', '뿌리', '상온', '가', '쓰오우', '동인', '분', '한', '뿌리', '인', '삼입', '일반', '상온', '맥스', '봉', '치즈', '농협', '년', '가', '쓰오우', '동인', '튀김', '우동', '인', '냉장', '즐거운', '동행', '콩', '칼국수', '냉장', '즐거운', '동행', '콩', '칼국수', '냉장', '다담', '부대찌개', '양념', '냉장', '백설', '만두', '

In [ ]:
# https://roboreport.co.kr/doc2vec-%ED%9B%88%EB%A0%A8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%EC%84%A4%EB%AA%85/
# 모델 설계
max_epochs = 3000

models = []
# 0 Class dummy
models.append(Doc2Vec(vector_size= 0, alpha=0, min_alpha=0, min_count=1,dm=1))

for n in range(1,6):
  model = Doc2Vec(vector_size= 500,
                  alpha=0.025, #learning rate
                  min_alpha=0.00025,
                  min_count=1, #데이터에서 등장하는 단어의 최소빈도수
                  dm=1)

  #build_vocab
  model.build_vocab(tagged_datas[n])

  # 학습
  for epoch in range(max_epochs):
      print("{}th Class's Epoch count : {} @@@@".format(n,epoch))
      model.train(tagged_datas[n],
                  total_examples=model.corpus_count,
                  epochs=model.iter)
      # decrease the learning rate
      model.alpha -= 0.0002
      # fix the learning rate, no decay
      model.min_alpha = model.alpha

  model.save("d2v_{}_3000_500_CJ.model".format(n))


1th Class's Epoch count : 0 @@@@
1th Class's Epoch count : 1 @@@@


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4th Class's Epoch count : 1000 @@@@
4th Class's Epoch count : 1001 @@@@
4th Class's Epoch count : 1002 @@@@
4th Class's Epoch count : 1003 @@@@
4th Class's Epoch count : 1004 @@@@
4th Class's Epoch count : 1005 @@@@
4th Class's Epoch count : 1006 @@@@
4th Class's Epoch count : 1007 @@@@
4th Class's Epoch count : 1008 @@@@
4th Class's Epoch count : 1009 @@@@
4th Class's Epoch count : 1010 @@@@
4th Class's Epoch count : 1011 @@@@
4th Class's Epoch count : 1012 @@@@
4th Class's Epoch count : 1013 @@@@
4th Class's Epoch count : 1014 @@@@
4th Class's Epoch count : 1015 @@@@
4th Class's Epoch count : 1016 @@@@
4th Class's Epoch count : 1017 @@@@
4th Class's Epoch count : 1018 @@@@
4th Class's Epoch count : 1019 @@@@
4th Class's Epoch count : 1020 @@@@
4th Class's Epoch count : 1021 @@@@
4th Class's Epoch count : 1022 @@@@
4th Class's Epoch count : 1023 @@@@
4th Class's Epoch count : 1024 @@@@
4th Class's Epoch count : 1025 @@@@
4th Class's Epoch count : 10

In [ ]:
# model 불러올때 실행
models = []
models.append('')
models.append(Doc2Vec.load('d2v_1_3000_500_CJ.model'))
models.append(Doc2Vec.load('d2v_2_3000_500_CJ.model'))
models.append(Doc2Vec.load('d2v_3_3000_500_CJ.model'))
models.append(Doc2Vec.load('d2v_4_3000_500_CJ.model'))
models.append(Doc2Vec.load('d2v_5_3000_500_CJ.model'))

In [ ]:
cor = 0
wro = 0
fxx = 0
for i in range(0,df_train.shape[0]):
#for i in range(0,2):
  # print('Calc similar : {}th'.format(i))
  try:
    sample = spacing_okt(df_train.loc[i, 'NM'])
    vectors = []
    vectors.append('')
    similar_docs = []
    similar_docs.append('')
    newCd = ""
    orgCd = ""
    # print(df_train.loc[i, 'NM'])
    # print(df_train.loc[i, 'CD'])
    for n in range(1,6):
      
      vectors.append(models[n].infer_vector(sample))
      similar_docs.append(models[n].docvecs.most_similar([vectors[n]], topn=10))
      orgCd = newCd
      if n == 1:
        newCd = labels[n][int(similar_docs[n][0][0])]
        continue
      for index, similarity in similar_docs[n]:
        #print("orgCd : "+orgCd)
        #print("labels[n][int(index)] : "+labels[n][int(index)])
        if orgCd in labels[n][int(index)]:
          newCd = labels[n][int(index)]
          break

      # 전 코드와 매칭되는 다음 코드가 없을 시, 넘어감
      if orgCd==newCd:
        fxx += 1
        break
      if n==5:
        print(df_train.loc[i, 'NM'])
        print(df_train.loc[i, 'CD'])
        print(newCd)
        if df_train.loc[i, 'CD'] == newCd:
          cor += 1
          print("corr")
        else:
          wro += 1
  except:
    pass

리치카라멜향시럽상온
01010060001200000024
02120320007600000152
백설고추장삼겹살구이양념상온
02130330007800000155
02050180004400000093
정면승부중화짜장면입용기
03170510011700000235
02050190004800000097
더건강한통오리오븐구이쌈무기획냉장
03150460010500000209
02100300007200000144
특별한선택호
04350780018400000341
01030140003700000069
실속탕수육
03230650016500000319
03150440010200000204
하선정알뜰형멸치액젓
02130350008300000165
02070210005500000116


In [ ]:
print(cor)
print(wro)
print(fxx)

0
0
2721


In [ ]:
cor = 0
cnt = 0
cSim = 0
nSim = 0
for i in range(0,df_train.shape[0]):
  # print('Calc similar : {}th'.format(i))
  try:
    sample = spacing_okt(df_train.loc[i, 'NM'])
    vector = models[1].infer_vector(sample)
    similar_docs = models[1].docvecs.most_similar([vector], topn=1)
    #if similar_docs[0][1] >= 0.65: #임시 정확도 65퍼만 뽑아본다
    for index, similarity in similar_docs:
      if labels[1][int(similar_docs[0][0])] in df_train.loc[i, 'CD']:
        #print(similar_docs[0][1])
        #print(sample)
        cSim +=  similar_docs[0][1]
        cor += 1;
      else:
        nSim +=  similar_docs[0][1]
    cnt+=1
  except:
    pass

쁘띠첼 밀감
쁘띠첼 밀감
볼로냐 스파게티인 남원 냉장
튀김 전용 유입 상온
햇반 작 은공 기입 상온
맛소금
통감자 바삭 부침가 루고급 상온
쁘띠첼 밀감
쁘띠첼 복숭아
허니 머스타드 소스
행복한 콩가지 국산 콩 찌개 냉장
가 쓰오 우동한 그릇 입 용기 냉장
햇반 큰 공기
쁘띠첼요 거 젤리 밀감
쁘띠첼요 거 젤리 딸기
조개 다시다
쇠고기 다시다 골드
백설 돼지 불고기 양념 상온
백설 돼지갈비 양념 상온
맥스 봉 치즈개 냉장
맥스 봉 치즈 낱개 냉장
다담 순두부찌개 양념 냉장
튀김 우동 인분 냉장
홈 조리 스파게티 인분 냉장
쁘띠첼요 거 젤리 복숭아
년가 쓰오우 동인분
가 쓰오우 동인분
맥스 봉 치즈 농협
즐거운 동행 콩 칼국수 냉장
쌀엿 상온
밥이랑 야채
밥이랑 해물
런 천 미트
햇반
숯불 구 이 맛 김밥 햄 냉장
런 천 미트
허브 솔트 스파이 시
파인애플 샐러드 소스 냉장
꽃 소금
구운소금
구운소금
구운소금
신 백설 너비아니 냉동
볼로냐 스파게티 인분 남원 냉장
백설군만 두 기획 중단 냉동
스팸 마 일드
쌍 백당
오리엔 탈 드레싱 냉장
하선정 다진 마늘 냉장
년가 쓰오우 동인분 냉장
양송이 덮 밥 소스
년가 쓰오우 동인 얼 큰우 동인
꽃 소금
남원 동치미 물냉면 인분 냉장
얼 큰우 동인분 냉장
치킨너겟 뉴
숯불 갈비 후랑 크 냉장
가 쓰 오메 밀생면 인분 냉장
가 쓰오 다시다 골드 스틱
스팸 고급 유호
햇반
쁘띠첼 워터 젤리 복숭아 냉장
정말 맛있는 소시지
콩기름 기획 상온
햇반 오곡밥 개입 상온
햇반 흑 미 밥
중력
신 강력 등
강력 등
중력 등
하선정 맛김치
팬 솔트 용기
햇반 찰 보리밥
구이 한판 냉장
홍 삼진
홍삼 진입
홍 삼진 골드
백설 햇 바삭 김재래 김소 포장 미사 용단
햇 바삭 김재래 김상 온
행복한 콩 행복한 콩 연 두부 용기 냉
태양 초고추장 상온
태양 초 골드 고추장 상온
태양 초 골드 고추장 봉상 온
태양 초 골드 고추장 상온
해찬들 태양 초 골드 고추장
구수한 집 된장
해찬들 태양 초 매운 고추장
사계절 쌈장 캔
사계절 쌈장
사계절 쌈장 

In [ ]:
print(cor)
print(cnt)
print(cSim)
print(nSim)

1238
2727
798.1134019300807
629.5440816879272
